## Reproduce Ershadi (2022)

A layered example with multiple azimuthal rotations, anisotropic reflectivity of varying magnitude, and birefringence of varying strength. Note that there is a small difference in how rotations are treated from the original publication so depths below 3 km have azimuthal variations from the original figure.

Ershadi, M. R., Drews, R., Mart´ın, C., Eisen, O., Ritz, C., Corr, H., . . . Mulvaney, R. (2022). Polarimetric radar reveals the spatial distribution of ice fabric at domes and divides in East Antarctica. The Cryosphere, 16 (5), 1719–1739. doi:10.5194/tc-16-1719-2022

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

from effmed.lib.matrix_model import effective_medium
from effmed.lib.supplemental import dB

from impdar.lib.ApresData.load_quadpol import load_quadpol_fujita

%matplotlib inline

In [ ]:
### Figure 3

# instantiate the model class
em = effective_medium()

# instrument properties
fc = 300e6
em.system_setup(fc)

# material properties
Temp = 253.
epsr = 3.15
em.epsr = epsr

# geometry
zs = np.linspace(.01,4000,4000)
psis = np.array([45,45,45,45,135,135,120, 120])*np.pi/180. + .0001
thetas = np.zeros(len(psis))
layer_dzs = 500*np.ones(len(psis))

# material anisotropy
dχs = np.array([.025, 0.2, 0.2, 0.2, 0.2, 0.45, 0.2, 0.2])
chis = np.empty((8,3))
for i in range(8):
    chis[i] = np.array([(.5-dχs[i])/2.,(.5+dχs[i])/2.,0.5])

# scattering matrices
gammaxs = 1e-12*np.ones_like(zs)
gammays = 1e-12*np.ones_like(zs)
gammays[zs>1000.] = 1e-11
gammays[zs>1500.] = 1e-13
gammays[zs>2500.] = 1e-14
gammays[zs>3000.] = 1e-12
gammas = np.transpose([gammaxs,gammays])

# rotation of scattering matrix
psi_gammas = np.pi/4.*np.ones(len(zs))
psi_gammas[zs>2000.] = 3.*np.pi/4.
psi_gammas[zs>3000.] = 120*np.pi/180.

# assign all the layer properties to the model class
em.ice_properties(idctx='biaxial',epsr=epsr,theta=thetas[0],psi=psis[0],chi=chis[0])

# solve the model
em.solve(zs,layer_dzs,thetas,psis,chis,gammas=gammas,psi_gammas=psi_gammas)

In [ ]:
# load the model output as an ImpDAR data object
dat = load_quadpol_fujita(em)
# rotate through all azimuths and make a grid
dat.rotational_transform(n_thetas=200)
Θs,Ds = np.meshgrid(dat.thetas,dat.range)
# find the cross-polarization extinction
dat.cpe_idxs = np.ones_like(dat.range).astype(int)
# get the co-polarization phase coherence
dat.coherence2d(delta_theta=.1,delta_range=1.1)
# phase gradient
dat.phase_gradient2d()

In [ ]:
### plot the model output following Figure 3 in Ershadi et al. (2022)

plt.figure(figsize=(12, 9))

# HH Power
plt.subplot(141)
HH = np.real(dB(dat.HH**2.))
HH = np.transpose(HH) - np.mean(HH,axis=1)
HH = np.transpose(HH)
plt.contourf(Θs,Ds,HH,cmap='hot',levels=np.arange(-15,15,.1),vmin=-15,vmax=15,extend='both')
plt.colorbar(label='Co-polarized (dB)',location='top',ticks=np.arange(-15,16,5))
plt.gca().invert_yaxis()
plt.xlabel('Azimuth')
plt.ylabel('Depth (m)')
plt.xticks([0,np.pi/2.,np.pi])
plt.gca().set_xticklabels(['0','π/2','π'])

plt.subplot(143)
plt.tick_params(labelleft=False)
HV = np.real(dB(dat.HV**2.))
HV = np.transpose(HV) - np.mean(HV,axis=1)
HV = np.transpose(HV)
plt.contourf(Θs,Ds,HV,cmap='hot',levels=np.arange(-15,15,.1),vmin=-15,vmax=15,extend='both')
plt.colorbar(label='Cross-polarized (dB)',location='top',ticks=np.arange(-15,16,5))
plt.gca().invert_yaxis()
plt.xlabel('Azimuth (rad)')
plt.xticks([0,np.pi/2.,np.pi])
plt.gca().set_xticklabels(['0','π/2','π'])

plt.subplot(142)
plt.tick_params(labelleft=False)
plt.contourf(Θs,Ds,np.angle(dat.chhvv),cmap='twilight_shifted',levels=100,vmin=-np.pi,vmax=np.pi)
cbar = plt.colorbar(label='$ϕ_{HHVV}$',location='top',ticks=np.array([-np.pi,0,np.pi]))
cbar.set_ticklabels(['-π','0','π'])
plt.gca().invert_yaxis()
plt.xlabel('Azimuth')
plt.xticks([0,np.pi/2.,np.pi])
plt.gca().set_xticklabels(['0','π/2','π'])

dϕ = dat.dphi_dz.copy()
clim = .05
dϕ[dϕ>clim]=clim
dϕ[dϕ<-clim]=-clim

plt.subplot(144)
plt.tick_params(labelleft=False)
plt.contourf(Θs,Ds,dϕ,cmap='seismic',levels=100,vmin=-clim,vmax=clim,extend='both')
plt.colorbar(label='$\partial ϕ/\partial z$',location='top',ticks=[-clim,0,clim])
plt.gca().invert_yaxis()
plt.xlabel('Azimuth')
plt.xticks([0,np.pi/2.,np.pi])
plt.gca().set_xticklabels(['0','π/2','π'])

plt.tight_layout()